In [ ]:
#Lab | Random Forests
#For this lab, you will be using the CSV files provided in the files_for_lab folder.

#Instructions

#1. Apply the Random Forests algorithm but this time only by upscaling the data using SMOTE.
#2. Note that since SMOTE works on numerical data only, we will first encode the categorical variables in this case.

In [34]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from imblearn.over_sampling import SMOTE

In [35]:
categorical=pd.read_csv(r'C:\Users\TOSHIBA\Downloads\Ironhack\Activities\Activities Python\Databases\categorical2.csv')
categorical.columns=[c.lower() for c in categorical.columns]
categorical.head()

,state,cluster,homeownr,gender,datasrce,rfa_2r,rfa_2a,geocode2,domain_a,domain_b,...,dob_yr,dob_mm,minrdate_yr,minrdate_mm,maxrdate_yr,maxrdate_mm,lastdate_yr,lastdate_mm,firstdate_yr,firstdate_mm
0,IL,36,H,F,3,L,E,C,T,2,...,37,12,92,8,94,2,95,12,89,11
1,CA,14,H,M,3,L,G,A,S,1,...,52,2,93,10,95,12,95,12,93,10
2,NC,43,U,M,3,L,E,C,R,2,...,0,2,91,11,92,7,95,12,90,1
3,CA,44,U,F,3,L,E,C,R,2,...,28,1,87,11,94,11,95,12,87,2
4,FL,16,H,F,3,L,F,A,S,2,...,20,1,93,10,96,1,96,1,79,3


In [65]:
numerical=pd.read_csv(r'C:\Users\TOSHIBA\Downloads\Ironhack\Activities\Activities Python\Databases\numerical2.csv')
numerical.columns=[c.lower() for c in numerical.columns]
numerical.head()

,tcode,age,income,wealth1,hit,malemili,malevet,vietvets,wwiivets,localgov,...,cardgift,minramnt,maxramnt,lastgift,timelag,avggift,controln,hphone_d,rfa_2f,cluster2
0,0,60.000000,5,9,0,0,39,34,18,10,...,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,...,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,...,14,2.0,16.0,5.0,12,7.481481,15078,1,4,60
3,0,70.000000,1,4,2,0,23,14,31,3,...,7,2.0,11.0,10.0,9,6.812500,172556,1,4,41
4,0,78.000000,3,2,60,1,28,9,53,26,...,8,3.0,15.0,15.0,14,6.864865,7112,1,2,26


In [37]:
target=pd.read_csv(r'C:\Users\TOSHIBA\Downloads\Ironhack\Activities\Activities Python\Databases\target2.csv')
target.columns=[c.lower() for c in target.columns]
target.head()

,target_b,target_d
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0


In [38]:
target['target_d'].value_counts()

0.00     90569
10.00      941
15.00      591
20.00      577
5.00       503
         ...  
18.25        1
10.70        1
2.50         1
16.87        1
44.21        1
Name: target_d, Length: 71, dtype: int64

In [39]:
print(categorical.isnull().sum())
print(numerical.isnull().sum())
print(target.isnull().sum())

state           0
cluster         0
homeownr        0
gender          0
datasrce        0
rfa_2r          0
rfa_2a          0
geocode2        0
domain_a        0
domain_b        0
odatew_yr       0
odatew_mm       0
dob_yr          0
dob_mm          0
minrdate_yr     0
minrdate_mm     0
maxrdate_yr     0
maxrdate_mm     0
lastdate_yr     0
lastdate_mm     0
firstdate_yr    0
firstdate_mm    0
dtype: int64
tcode       0
age         0
income      0
wealth1     0
hit         0
           ..
avggift     0
controln    0
hphone_d    0
rfa_2f      0
cluster2    0
Length: 315, dtype: int64
target_b    0
target_d    0
dtype: int64


In [46]:
#Vamos a tratar los datos categoricos
from sklearn import preprocessing
labels=categorical.apply(preprocessing.LabelEncoder().fit_transform)
labels.head()

,state,cluster,homeownr,gender,datasrce,rfa_2r,rfa_2a,geocode2,domain_a,domain_b,...,dob_yr,dob_mm,minrdate_yr,minrdate_mm,maxrdate_yr,maxrdate_mm,lastdate_yr,lastdate_mm,firstdate_yr,firstdate_mm
0,3,35,0,0,2,0,1,2,3,1,...,36,11,14,7,14,1,0,11,18,10
1,0,13,0,1,2,0,3,0,2,0,...,51,1,15,9,15,11,0,11,22,9
2,7,42,1,1,2,0,1,2,1,1,...,0,1,13,10,12,6,0,11,19,0
3,0,43,1,0,2,0,1,2,1,1,...,27,0,9,10,14,10,0,11,16,1
4,1,15,0,0,2,0,2,0,2,1,...,19,0,15,9,16,0,1,0,8,2


In [66]:
#Concatenamos la nueva tabla con datos numericos junto con la tabla numerica original
conc=np.concatenate((numerical,labels),axis=1)
conc=pd.DataFrame(conc)
conc.head()

,0,1,2,3,4,5,6,7,8,9,...,327,328,329,330,331,332,333,334,335,336
0,0.0,60.000000,5.0,9.0,0.0,0.0,39.0,34.0,18.0,10.0,...,36.0,11.0,14.0,7.0,14.0,1.0,0.0,11.0,18.0,10.0
1,1.0,46.000000,6.0,9.0,16.0,0.0,15.0,55.0,11.0,6.0,...,51.0,1.0,15.0,9.0,15.0,11.0,0.0,11.0,22.0,9.0
2,1.0,61.611649,3.0,1.0,2.0,0.0,20.0,29.0,33.0,6.0,...,0.0,1.0,13.0,10.0,12.0,6.0,0.0,11.0,19.0,0.0
3,0.0,70.000000,1.0,4.0,2.0,0.0,23.0,14.0,31.0,3.0,...,27.0,0.0,9.0,10.0,14.0,10.0,0.0,11.0,16.0,1.0
4,0.0,78.000000,3.0,2.0,60.0,1.0,28.0,9.0,53.0,26.0,...,19.0,0.0,15.0,9.0,16.0,0.0,1.0,0.0,8.0,2.0


In [68]:
#Definimos X y Y
X=conc
y=target['target_b']

In [70]:
smote=SMOTE()
y=target['target_b']
X=conc
X_sm,y_sm=smote.fit_resample(X,y)
y_sm.value_counts()

0    90569
1    90569
Name: target_b, dtype: int64

In [71]:
#Hacemos el split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.3, random_state=40)

In [73]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [80, 150],
    'min_samples_split': [2, 4],
    'min_samples_leaf' : [1, 2],
    'max_features': ['sqrt']

    }
clf = RandomForestClassifier(random_state=100)

grid_search = GridSearchCV(clf, param_grid, cv=5,return_train_score=True,n_jobs=-1)
grid_search.fit(X_train,y_train)
grid_search.best_params_ #To check the best set of parameters returned

{'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 150}

In [74]:
#Importamos RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(random_state=0, max_features='sqrt', min_samples_leaf=1, min_samples_split=4, n_estimators=100)
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(cross_val_scores))

0.9733824634390109


In [30]:
#Para eliminar columnas, ubicamos las que tengan un p-value cercano a 1 con el metodo OLS
#X=sm.add_constant(conc)
#model=sm.OLS(y,X).fit()
#model.summary()

In [27]:
#model.pvalues

In [28]:
#Funcion para obtener una lista del p-value
#d = {}
#for i in X.columns.tolist():
    #d[f'{i}'] = model.pvalues[i]

#df_pvalue= pd.DataFrame(d.items(), columns=['Var_name', 'p-Value']).reset_index(drop=True)
#df_pvalue

In [29]:
#Vamos a poner en una lista alternativa qué columnas no cumplen 
#con un máximo de nulos propuestos. Supongamos que nuestro umbral/threshold es de .7
#columns_above_threshold=df_pvalue[df_pvalue['p-Value']>0.7]
#print(columns_above_threshold)